In [27]:
import pandas as pd
from sklearn import svm

path = 'datasets/drug_consumption_engineered.csv'
dataset_D = pd.read_csv(path)

dataset_D.head()

,ID,Age,Gender,Education,Countery,Ethnicity,NScore,EScore,OScore,AScore,...,Cannabis,Cannabis_Consume,Choc,Choc_Consume,Ecstasy,Ecstasy_Consume,Nicotine,Nicotine_Consume,VSA,VSA_Consume
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,...,CL0,0,CL5,1,CL0,0,CL2,1,CL0,0
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,...,CL4,1,CL6,1,CL4,1,CL4,1,CL0,0
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,...,CL3,1,CL4,1,CL0,0,CL0,0,CL0,0
3,4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,...,CL2,1,CL4,1,CL0,0,CL2,1,CL0,0
4,5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,...,CL3,1,CL6,1,CL1,0,CL2,1,CL0,0


In [28]:

selected_feature_columns = ['Age', 'Gender', 'Education', 'Countery', 'Ethnicity', 'NScore', 'EScore', 'OScore', 'AScore', 'CScore', 'Impulsive', 'SS']
selected_feature = dataset_D.loc[:, selected_feature_columns]
selected_feature.head()

,Age,Gender,Education,Countery,Ethnicity,NScore,EScore,OScore,AScore,CScore,Impulsive,SS
0,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084
1,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575
2,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148
3,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084
4,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575


In [29]:
selected_target = dataset_D.Cannabis_Consume
selected_target.head()

0    0
1    1
2    1
3    1
4    1
Name: Cannabis_Consume, dtype: int64

In [30]:
# Source: https://www.kaggle.com/code/pbizil/machine-learning-models-and-friedman-test/notebook
def ranking_model(results_aggregate):
    ranking = pd.DataFrame(columns=results_aggregate.columns)
    for i in range(results_aggregate.shape[0]):
        ranking.loc[i, results_aggregate.iloc[i].rank(ascending=False).index]=results_aggregate.iloc[i].rank(ascending=False)
    return ranking

In [31]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing

imbalanced_results = pd.DataFrame()

from sklearn import tree 
dt_classifer = make_pipeline(preprocessing.MinMaxScaler(),tree.DecisionTreeClassifier())
dt_scores = cross_val_score(dt_classifer, selected_feature, selected_target, cv=10)
imbalanced_results["dt_scores"] = dt_scores 

from sklearn.neighbors import KNeighborsClassifier
knn_classifer = make_pipeline(preprocessing.MinMaxScaler(), KNeighborsClassifier())
knn_scores = cross_val_score(knn_classifer, selected_feature, selected_target, cv=10)
imbalanced_results["knn_scores"] = knn_scores 

from sklearn.ensemble import RandomForestClassifier
rf_classifer = make_pipeline(preprocessing.MinMaxScaler(), RandomForestClassifier())
rf_scores = cross_val_score(rf_classifer, selected_feature, selected_target, cv=10)
imbalanced_results["rf_scores"] = rf_scores 

from sklearn import svm
svm_classifer = make_pipeline(preprocessing.MinMaxScaler(), svm.SVC())
svm_scores = cross_val_score(svm_classifer, selected_feature, selected_target, cv=10)
imbalanced_results["svm_scores"] = svm_scores 

from sklearn.neural_network import MLPClassifier
mlp_classifer = make_pipeline(preprocessing.MinMaxScaler(), MLPClassifier(max_iter=600))
mlp_scores = cross_val_score(mlp_classifer, selected_feature, selected_target, cv=10)
imbalanced_results["mlp_scores"] = mlp_scores 

from sklearn.ensemble import GradientBoostingClassifier
gb_classifer = make_pipeline(preprocessing.MinMaxScaler(), GradientBoostingClassifier())
gb_scores = cross_val_score(gb_classifer, selected_feature, selected_target, cv=10)
imbalanced_results["gb_scores"] = gb_scores 

ranks = ranking_model(imbalanced_results)
print(ranks)


  dt_scores knn_scores rf_scores svm_scores mlp_scores gb_scores
0       6.0        5.0       4.0        1.0        3.0       2.0
1       6.0        5.0       3.0        1.0        4.0       2.0
2       6.0        5.0       2.0        4.0        3.0       1.0
3       6.0        5.0       3.5        1.5        1.5       3.5
4       6.0        4.0       5.0        2.0        1.0       3.0
5       6.0        5.0       2.5        2.5        1.0       4.0
6       6.0        5.0       4.0        3.0        1.0       2.0
7       6.0        3.5       2.0        1.0        3.5       5.0
8       6.0        4.0       1.5        1.5        5.0       3.0
9       6.0        5.0       1.5        1.5        3.0       4.0


In [32]:
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn import metrics

db1_kfold = dataset_D.copy(deep=True)

selected_feature_columns = ['Age', 'Gender', 'Education', 'Countery', 'Ethnicity', 'NScore', 'EScore', 'OScore', 'AScore', 'CScore', 'Impulsive', 'SS']
selected_feature = db1_kfold.loc[:, selected_feature_columns]
selected_target = db1_kfold.Cannabis_Consume

preprocessing.MinMaxScaler()
selected_feature = preprocessing.MinMaxScaler().fit_transform(selected_feature)

from sklearn import tree 
dt_classifer_os = tree.DecisionTreeClassifier()

from sklearn.neighbors import KNeighborsClassifier
knn_classifer_os = KNeighborsClassifier()

from sklearn.ensemble import RandomForestClassifier
rf_classifer_os =  RandomForestClassifier()

from sklearn import svm
svm_classifer_os = svm.SVC()

from sklearn.neural_network import MLPClassifier
mlp_classifer_os = MLPClassifier(max_iter=600)

from sklearn.ensemble import GradientBoostingClassifier
gb_classifer_os = GradientBoostingClassifier()

k_fold_cv_os = KFold(n_splits=10)
smote_os = SMOTE()

dt_scores_os = []
knn_scores_os = []
rf_scores_os = []
svm_scores_os = []
mlp_scores_os = []
gb_scores_os = []
results_os = pd.DataFrame()

for train_index, test_index in k_fold_cv_os.split(selected_feature, selected_target):
    train_selected_feature_fold, train_selected_target_fold = selected_feature[train_index], selected_target[train_index]
    test_selected_feature_fold, test_selected_target_fold = selected_feature[test_index], selected_target[test_index]

    train_selected_feature_os, train_selected_target_os = smote_os.fit_resample(train_selected_feature_fold, train_selected_target_fold)

    dt_model_os = dt_classifer_os.fit(train_selected_feature_os, train_selected_target_os)
    knn_model_os = knn_classifer_os.fit(train_selected_feature_os, train_selected_target_os)
    rf_model_os = rf_classifer_os.fit(train_selected_feature_os, train_selected_target_os)
    svm_model_os = svm_classifer_os.fit(train_selected_feature_os, train_selected_target_os)
    mlp_model_os = mlp_classifer_os.fit(train_selected_feature_os, train_selected_target_os)
    gb_model_os = gb_classifer_os.fit(train_selected_feature_os, train_selected_target_os)
    
    dt_scores_os.append(metrics.recall_score(test_selected_target_fold, dt_model_os.predict(test_selected_feature_fold)))
    knn_scores_os.append(metrics.recall_score(test_selected_target_fold, knn_model_os.predict(test_selected_feature_fold)))
    rf_scores_os.append(metrics.recall_score(test_selected_target_fold, rf_model_os.predict(test_selected_feature_fold)))
    svm_scores_os.append(metrics.recall_score(test_selected_target_fold, svm_model_os.predict(test_selected_feature_fold)))
    mlp_scores_os.append(metrics.recall_score(test_selected_target_fold, mlp_model_os.predict(test_selected_feature_fold)))
    gb_scores_os.append(metrics.recall_score(test_selected_target_fold, gb_model_os.predict(test_selected_feature_fold)))

results_os["dt_scores_os"] = dt_scores_os
results_os["knn_scores_os"] = knn_scores_os
results_os["rf_scores_os"] = rf_scores_os
results_os["svm_scores_os"] = svm_scores_os
results_os["mlp_scores_os"] = mlp_scores_os
results_os["gb_scores_os"] = gb_scores_os

ranks_os = ranking_model(results_os)
print(ranks_os)

/Users/mohamed/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mohamed/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mohamed/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mohamed/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/m

  dt_scores_os knn_scores_os rf_scores_os svm_scores_os mlp_scores_os  \
0          1.5           6.0          1.5           4.0           5.0   
1          1.0           4.0          3.0           5.0           6.0   
2          3.0           4.5          1.0           6.0           4.5   
3          3.0           6.0          1.0           5.0           4.0   
4          6.0           4.0          1.5           3.0           5.0   
5          6.0           5.0          1.0           4.0           3.0   
6          5.0           6.0          1.0           3.0           3.0   
7          3.0           6.0          1.0           4.0           5.0   
8          1.5           6.0          3.0           4.0           5.0   
9          5.0           6.0          1.0           4.0           3.0   

  gb_scores_os  
0          3.0  
1          2.0  
2          2.0  
3          2.0  
4          1.5  
5          2.0  
6          3.0  
7          2.0  
8          1.5  
9          2.0  


In [33]:
from sklearn.model_selection import KFold
from imblearn.under_sampling import EditedNearestNeighbours, ClusterCentroids
from sklearn import metrics

db2_kfold = dataset_D.copy(deep=True)

selected_feature_columns = ['Age', 'Gender', 'Education', 'Countery', 'Ethnicity', 'NScore', 'EScore', 'OScore', 'AScore', 'CScore', 'Impulsive', 'SS']
selected_feature = db2_kfold.loc[:, selected_feature_columns]
selected_target = db2_kfold.Cannabis_Consume

preprocessing.MinMaxScaler()
selected_feature = preprocessing.MinMaxScaler().fit_transform(selected_feature)

from sklearn import tree 
dt_classifer_us = tree.DecisionTreeClassifier()

from sklearn.neighbors import KNeighborsClassifier
knn_classifer_us = KNeighborsClassifier()

from sklearn.ensemble import RandomForestClassifier
rf_classifer_us =  RandomForestClassifier()

from sklearn import svm
svm_classifer_us = svm.SVC()

from sklearn.neural_network import MLPClassifier
mlp_classifer_us = MLPClassifier(max_iter=600)

from sklearn.ensemble import GradientBoostingClassifier
gb_classifer_us = GradientBoostingClassifier()

k_fold_cv_us = KFold(n_splits=10)
enn_us = EditedNearestNeighbours() #ClusterCentroids() 

dt_scores_us = []
knn_scores_us = []
rf_scores_us = []
svm_scores_us = []
mlp_scores_us = []
gb_scores_us = []
results_us = pd.DataFrame()

for train_index, test_index in k_fold_cv_us.split(selected_feature, selected_target):
    train_selected_feature_fold, train_selected_target_fold = selected_feature[train_index], selected_target[train_index]
    test_selected_feature_fold, test_selected_target_fold = selected_feature[test_index], selected_target[test_index]

    train_selected_feature_us, train_selected_target_us = enn_us.fit_resample(train_selected_feature_fold, train_selected_target_fold)

    dt_model_us = dt_classifer_us.fit(train_selected_feature_us, train_selected_target_us)
    knn_model_us = knn_classifer_us.fit(train_selected_feature_us, train_selected_target_us)
    rf_model_us = rf_classifer_us.fit(train_selected_feature_us, train_selected_target_us)
    svm_model_us = svm_classifer_us.fit(train_selected_feature_us, train_selected_target_us)
    mlp_model_us = mlp_classifer_us.fit(train_selected_feature_us, train_selected_target_us)
    gb_model_us = gb_classifer_us.fit(train_selected_feature_us, train_selected_target_us)
    
    dt_scores_us.append(metrics.recall_score(test_selected_target_fold, dt_model_us.predict(test_selected_feature_fold)))
    knn_scores_us.append(metrics.recall_score(test_selected_target_fold, knn_model_us.predict(test_selected_feature_fold)))
    rf_scores_us.append(metrics.recall_score(test_selected_target_fold, rf_model_us.predict(test_selected_feature_fold)))
    svm_scores_us.append(metrics.recall_score(test_selected_target_fold, svm_model_us.predict(test_selected_feature_fold)))
    mlp_scores_us.append(metrics.recall_score(test_selected_target_fold, mlp_model_us.predict(test_selected_feature_fold)))
    gb_scores_us.append(metrics.recall_score(test_selected_target_fold, gb_model_us.predict(test_selected_feature_fold)))

results_us["dt_scores_us"] = dt_scores_us
results_us["knn_scores_us"] = knn_scores_us
results_us["rf_scores_us"] = rf_scores_us
results_us["svm_scores_us"] = svm_scores_us
results_us["mlp_scores_us"] = mlp_scores_us
results_us["gb_scores_us"] = gb_scores_us

ranks_us = ranking_model(results_us)
print(ranks_us)



/Users/mohamed/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mohamed/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mohamed/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mohamed/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/m

  dt_scores_us knn_scores_us rf_scores_us svm_scores_us mlp_scores_us  \
0          1.0           4.0          2.5           5.5           5.5   
1          1.0           2.5          4.5           4.5           6.0   
2          3.0           1.0          4.0           5.0           2.0   
3          1.0           2.0          4.0           5.5           3.0   
4          6.0           1.0          2.5           4.5           2.5   
5          6.0           5.0          3.5           1.0           3.5   
6          6.0           3.0          1.0           5.0           3.0   
7          6.0           4.0          1.0           3.0           5.0   
8          6.0           5.0          1.5           4.0           3.0   
9          6.0           4.0          2.0           3.0           5.0   

  gb_scores_us  
0          2.5  
1          2.5  
2          6.0  
3          5.5  
4          4.5  
5          2.0  
6          3.0  
7          2.0  
8          1.5  
9          1.0  


In [34]:
print(imbalanced_results["dt_scores"].mean(), results_os["dt_scores_os"].mean(), results_us["dt_scores_us"].mean())
print(imbalanced_results["knn_scores"].mean(), results_os["knn_scores_os"].mean(), results_us["knn_scores_us"].mean())
print(imbalanced_results["rf_scores"].mean(), results_os["rf_scores_os"].mean(), results_us["rf_scores_us"].mean())
print(imbalanced_results["svm_scores"].mean(), results_os["svm_scores_os"].mean(), results_us["svm_scores_us"].mean())
print(imbalanced_results["mlp_scores"].mean(), results_os["mlp_scores_os"].mean(), results_us["mlp_scores_us"].mean())
print(imbalanced_results["gb_scores"].mean(), results_os["gb_scores_os"].mean(), results_us["gb_scores_us"].mean())

0.7268490374873353 0.7687106190239736 0.6481528549959136
0.7793847799166947 0.6908510507859922 0.6615983988352859
0.8006050883710458 0.8019501590795786 0.6708058324828181
0.8101345266238884 0.7155127749589717 0.6561432875637305
0.8064308229201845 0.7050469719984341 0.6528711349291847
0.8042862771586176 0.7887635364296044 0.6663197951137274
